In [413]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import normalize

## Wczytanie danych treningowych

In [610]:
traindf = pd.read_csv("train.csv")
traindf_complete = traindf.loc[:,['ora',
#                               'irr_pvgis_mod','irri_pvgis_mod',
                                 'temperatura_ambiente',
                                 'irradiamento',
                                 'pressure', 'windspeed',
                                 'cloudcover',
                                 'kwh']]

X = traindf_complete.drop(['kwh'], axis=1)
Y = traindf_complete['kwh']

X.shape

(235790, 6)

## //risercz algorytmów

In [642]:
x_train,x_test,y_train,y_test = train_test_split(X,Y, test_size=0.4)

y_test.head(10)

169565    0.000
13544     0.000
124579    0.031
111285    0.000
54844     0.059
181757    0.464
156355    0.000
96045     0.000
63077     0.248
171664    0.677
Name: kwh, dtype: float64

In [643]:
# %%time
# # clf = svm.SVR(C=1, gamma=0.1, epsilon=0.1).fit(x_train, y_train)

# lss = Lasso(random_state=0, max_iter=1000)
# aaa = np.logspace(-10, -5, 30)
# par = [{'alpha': alphas}]
# nnn = 3

# clf = GridSearchCV(lss, par, cv=nnn, refit=True)
# clf.fit(x_train,y_train)

clf = RandomForestRegressor(max_depth = 40, n_estimators=400, n_jobs=4, random_state=0)

clf.fit(X, Y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=40,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=4,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [644]:
clf.score(x_test, y_test)
# clf.best_score_

0.98257660947250836

In [645]:
# clf.best_params_

In [646]:
from sklearn.metrics import mean_squared_error
yyy = clf.predict(x_test)
mean_squared_error(y_test, yyy)

0.00077237212715863953

In [647]:
yyy[:10]

array([  5.27674764e-06,   5.61252715e-06,   2.93700000e-02,
         7.56973209e-06,   4.77075000e-02,   4.64000017e-01,
         9.02688854e-04,   4.65412678e-06,   2.91425000e-01,
         6.75470000e-01])

## Wczytanie danych testowych

In [629]:
testdf = pd.read_csv("test.csv")
testdf_complete = testdf.loc[:,['ora',
#                                          'irr_pvgis_mod','irri_pvgis_mod',
                                         'temperatura_ambiente',
                                 'irradiamento',
                                 'pressure', 'windspeed',
                                 'cloudcover'
                               ]]

testdf_complete.head(3)

,ora,temperatura_ambiente,irradiamento,pressure,windspeed,cloudcover
0,0.0,0.212,0.0,0.0,0.041,0.00
1,0.0,0.212,0.0,0.0,0.001,0.00
2,0.0,0.212,0.0,0.0,0.021,0.31


## Uczenie

In [630]:
%%time
# modelfit = svm.SVR(C=1, gamma=1, epsilon=0.1).fit(X, Y)

# lasso = Lasso(random_state=0, max_iter=1000, normalize = True)
# alphas = np.logspace(-10, -5, 30)
# tuned_parameters = [{'alpha': alphas}]
# n_folds = 3

# modelfit = GridSearchCV(lasso, tuned_parameters, cv=n_folds, refit=True)


# model = LinearRegression()
# parameters = {'fit_intercept':[True,False], 'normalize':[True], 'copy_X':[True, False]}
# modelfit = GridSearchCV(model,parameters, cv=3)

modelfit = RandomForestRegressor(max_depth = 20, n_estimators=200, n_jobs=4, random_state=0)

# param_grid = { 
#     'n_estimators': [150, 200]
# }

# modelfit = GridSearchCV(rf,param_grid, cv=3)

modelfit.fit(X, Y)

Wall time: 1min 14s


In [631]:
# modelfit.best_params_

## Predykcja

In [632]:
%%time
pred_result = modelfit.predict(testdf_complete)
# pred_norm = normalize(pred_result[:,np.newaxis], axis=0).ravel()

Wall time: 926 ms


In [633]:
#  norm2[:10]

In [634]:
 testdf_complete[140:150]

,ora,temperatura_ambiente,irradiamento,pressure,windspeed,cloudcover
140,0.444,0.394,0.026,0.756,0.039,0.97
141,0.444,0.409,0.056,0.756,0.038,0.97
142,0.444,0.424,0.098,0.757,0.067,0.75
143,0.444,0.394,0.051,0.756,0.038,0.97
144,0.444,0.364,0.035,0.756,0.038,0.97
145,0.444,0.379,0.035,0.756,0.039,0.97
146,0.444,0.364,0.054,0.756,0.040,0.97
147,0.444,0.318,0.140,0.000,0.021,0.31
148,0.444,0.409,0.046,0.757,0.043,0.96
149,0.444,0.333,0.021,0.000,0.036,0.00


## Przygotowanie wyniku

In [635]:
norm = pred_result - min(pred_result)
max(norm)

0.70236166252715859

In [636]:
kwh = pd.Series(pred_result)
non_negative = np.round(pred_result*1000)/1000.0 #- min(pred_result)
non_negative[non_negative < 0] = 0
d = {'Id': testdf['id'].tolist(), 'kwh': non_negative}
result = pd.DataFrame(data=d)

## Zapis do csv

In [637]:
result.to_csv('wynik.csv',index=False)

In [638]:
result[140:150]

,Id,kwh
140,34735,0.045
141,34743,0.088
142,34776,0.319
143,34901,0.087
144,34949,0.055
145,34970,0.057
146,35111,0.098
147,35201,0.461
148,35260,0.096
149,35370,0.568


In [639]:
result[0:20]

,Id,kwh
0,34255,0.000
1,34282,0.000
2,34375,0.000
3,34572,0.000
4,34660,0.005
5,34761,0.000
6,34814,0.000
7,34888,0.000
8,34913,0.000
9,34987,0.000


In [640]:
max(non_negative)

0.70199999999999996

In [641]:
max(pred_result)

0.70236166252715859